In [1]:
import pandas as pd
import sqlite3
import json
import pyodbc
from datetime import datetime

In [2]:
customer_file = 'D:\Depi\Projects\ETL\data\Customers.csv'
product_file = 'D:\Depi\Projects\ETL\data\products.db'
order_file = 'D:\Depi\Projects\ETL\data\orders.json'

In [3]:
customer_df = pd.read_csv(customer_file)

In [4]:
customer_df

,customer_id,name,gender,city
0,101,Ahmed Ali,M,Cairo
1,102,Sara Hassan,F,Alexandria
2,103,Omar Saeed,M,Giza
3,104,Laila Mostafa,F,Cairo
4,105,Hassan Ibrahim,M,Alexandria
5,106,Mona Adel,F,Giza
6,107,Tarek Hussein,M,Cairo
7,108,Nour Youssef,F,Alexandria
8,109,Karim Farouk,M,Giza
9,110,Dina Mohamed,F,Cairo


In [5]:
conn = sqlite3.connect(product_file)
cursor = conn.cursor()

cursor.execute('''select * from products''')
Product_data = cursor.fetchall()
product_df = pd.DataFrame(Product_data ,  columns=['product_id', 'produvr_name','category' ,'price'])

cursor.close()
conn.close()

In [6]:
product_df

,product_id,produvr_name,category,price
0,201,Laptop,Electronics,150.0
1,202,Smartphone,Electronics,300.0
2,203,Desk Chair,Furniture,100.0
3,204,Tablet,Electronics,200.0
4,205,Headphones,Electronics,50.0
5,206,Office Desk,Furniture,250.0
6,207,Refrigerator,Appliances,500.0
7,208,Microwave,Appliances,120.0
8,209,Sofa,Furniture,400.0
9,210,Coffee Machine,Appliances,180.0


In [7]:
with open(order_file, 'r') as f:
    order_data = json.load(f)

order_data 

[{'order_id': 1,
  'customer_id': 101,
  'product_id': 201,
  'quantity': 2,
  'date': '2024-05-10'},
 {'order_id': 2,
  'customer_id': 102,
  'product_id': 202,
  'quantity': 1,
  'date': '2024-05-11'},
 {'order_id': 3,
  'customer_id': 103,
  'product_id': 203,
  'quantity': 3,
  'date': '2024-05-12'},
 {'order_id': 4,
  'customer_id': 104,
  'product_id': 204,
  'quantity': 1,
  'date': '2024-06-01'},
 {'order_id': 5,
  'customer_id': 105,
  'product_id': 205,
  'quantity': 4,
  'date': '2024-06-05'},
 {'order_id': 6,
  'customer_id': 106,
  'product_id': 206,
  'quantity': 2,
  'date': '2024-06-10'},
 {'order_id': 7,
  'customer_id': 107,
  'product_id': 207,
  'quantity': 1,
  'date': '2024-07-02'},
 {'order_id': 8,
  'customer_id': 108,
  'product_id': 208,
  'quantity': 2,
  'date': '2024-07-05'},
 {'order_id': 9,
  'customer_id': 109,
  'product_id': 209,
  'quantity': 1,
  'date': '2024-07-15'},
 {'order_id': 10,
  'customer_id': 110,
  'product_id': 210,
  'quantity': 3,
  'd

In [8]:
order_df = pd.DataFrame(order_data)

In [20]:
order_df

,order_id,customer_id,product_id,quantity,date,Date
0,1,101,201,2,2024-05-10,2024-05-10
1,2,102,202,1,2024-05-11,2024-05-11
2,3,103,203,3,2024-05-12,2024-05-12
3,4,104,204,1,2024-06-01,2024-06-01
4,5,105,205,4,2024-06-05,2024-06-05
5,6,106,206,2,2024-06-10,2024-06-10
6,7,107,207,1,2024-07-02,2024-07-02
7,8,108,208,2,2024-07-05,2024-07-05
8,9,109,209,1,2024-07-15,2024-07-15
9,10,110,210,3,2024-07-20,2024-07-20


In [10]:
order_df['Date'] = pd.to_datetime(order_df['date'])
order_df['Date']

0   2024-05-10
1   2024-05-11
2   2024-05-12
3   2024-06-01
4   2024-06-05
5   2024-06-10
6   2024-07-02
7   2024-07-05
8   2024-07-15
9   2024-07-20
Name: Date, dtype: datetime64[ns]

In [11]:
dim_date = order_df[['Date']].drop_duplicates().reset_index(drop=True)
dim_date

,Date
0,2024-05-10
1,2024-05-11
2,2024-05-12
3,2024-06-01
4,2024-06-05
5,2024-06-10
6,2024-07-02
7,2024-07-05
8,2024-07-15
9,2024-07-20


In [12]:
dim_date['DateKey'] = dim_date['Date'].dt.strftime('%Y%m%d').astype(int)
dim_date['DateKey'] 

0    20240510
1    20240511
2    20240512
3    20240601
4    20240605
5    20240610
6    20240702
7    20240705
8    20240715
9    20240720
Name: DateKey, dtype: int64

In [13]:
dim_date[['Year','Month','Day']] = dim_date['Date'].dt.strftime('%Y-%m-%d').str.split('-', expand=True).astype(int)


In [14]:
dim_date

,Date,DateKey,Year,Month,Day
0,2024-05-10,20240510,2024,5,10
1,2024-05-11,20240511,2024,5,11
2,2024-05-12,20240512,2024,5,12
3,2024-06-01,20240601,2024,6,1
4,2024-06-05,20240605,2024,6,5
5,2024-06-10,20240610,2024,6,10
6,2024-07-02,20240702,2024,7,2
7,2024-07-05,20240705,2024,7,5
8,2024-07-15,20240715,2024,7,15
9,2024-07-20,20240720,2024,7,20


In [28]:
fact_df = order_df.merge(order_df[['customer_id']], on='customer_id', how='left')
fact_df = fact_df.merge(product_df[['product_id']], on='product_id', how='left')
fact_df = fact_df.merge(dim_date[['Date', 'DateKey']], on='Date', how='left')

fact_df

,order_id,customer_id,product_id,quantity,date,Date,DateKey
0,1,101,201,2,2024-05-10,2024-05-10,20240510
1,2,102,202,1,2024-05-11,2024-05-11,20240511
2,3,103,203,3,2024-05-12,2024-05-12,20240512
3,4,104,204,1,2024-06-01,2024-06-01,20240601
4,5,105,205,4,2024-06-05,2024-06-05,20240605
5,6,106,206,2,2024-06-10,2024-06-10,20240610
6,7,107,207,1,2024-07-02,2024-07-02,20240702
7,8,108,208,2,2024-07-05,2024-07-05,20240705
8,9,109,209,1,2024-07-15,2024-07-15,20240715
9,10,110,210,3,2024-07-20,2024-07-20,20240720


In [30]:
fact_df = fact_df[['customer_id', 'product_id', 'quantity', 'DateKey']]
fact_df

,customer_id,product_id,quantity,DateKey
0,101,201,2,20240510
1,102,202,1,20240511
2,103,203,3,20240512
3,104,204,1,20240601
4,105,205,4,20240605
5,106,206,2,20240610
6,107,207,1,20240702
7,108,208,2,20240705
8,109,209,1,20240715
9,110,210,3,20240720


In [31]:
fact_df = fact_df.astype({
    'customer_id': int,
    'product_id': int,
    'quantity': int,
    'DateKey': int
})

In [57]:
import pyodbc

server = 'localhost\SQLEXPRESS'
database = 'CustomerDW'
username = 'sa'
password = 'sa'
driver = '{ODBC Driver 17 for SQL Server}'

conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')

In [58]:
customer_df

,customer_id,name,gender,city
0,101,Ahmed Ali,M,Cairo
1,102,Sara Hassan,F,Alexandria
2,103,Omar Saeed,M,Giza
3,104,Laila Mostafa,F,Cairo
4,105,Hassan Ibrahim,M,Alexandria
5,106,Mona Adel,F,Giza
6,107,Tarek Hussein,M,Cairo
7,108,Nour Youssef,F,Alexandria
8,109,Karim Farouk,M,Giza
9,110,Dina Mohamed,F,Cairo


In [59]:
customers_data = customer_df[['customer_id', 'name', 'gender', 'city']].astype(str).values.tolist()
customers_data

[['101', 'Ahmed Ali', 'M', 'Cairo'],
 ['102', 'Sara Hassan', 'F', 'Alexandria'],
 ['103', 'Omar Saeed', 'M', 'Giza'],
 ['104', 'Laila Mostafa', 'F', 'Cairo'],
 ['105', 'Hassan Ibrahim', 'M', 'Alexandria'],
 ['106', 'Mona Adel', 'F', 'Giza'],
 ['107', 'Tarek Hussein', 'M', 'Cairo'],
 ['108', 'Nour Youssef', 'F', 'Alexandria'],
 ['109', 'Karim Farouk', 'M', 'Giza'],
 ['110', 'Dina Mohamed', 'F', 'Cairo']]

In [60]:
products_data = product_df[['product_id', 'produvr_name', 'category', 'price']].astype(object).values.tolist()
dates_data = dim_date[['DateKey', 'Date', 'Year', 'Month', 'Day']].copy()
dates_data['Date'] = dates_data['Date'].dt.date
dates_data = dates_data.values.tolist()
fact_data = fact_df.values.tolist()

In [61]:
cursor = conn.cursor()

cursor.executemany(
    "INSERT INTO dim_product (product_id, product_name, category, price) VALUES (?, ?, ?, ?)",
    Product_data
)


In [63]:
conn.commit()

In [64]:
dates_data

[[20240510, datetime.date(2024, 5, 10), 2024, 5, 10],
 [20240511, datetime.date(2024, 5, 11), 2024, 5, 11],
 [20240512, datetime.date(2024, 5, 12), 2024, 5, 12],
 [20240601, datetime.date(2024, 6, 1), 2024, 6, 1],
 [20240605, datetime.date(2024, 6, 5), 2024, 6, 5],
 [20240610, datetime.date(2024, 6, 10), 2024, 6, 10],
 [20240702, datetime.date(2024, 7, 2), 2024, 7, 2],
 [20240705, datetime.date(2024, 7, 5), 2024, 7, 5],
 [20240715, datetime.date(2024, 7, 15), 2024, 7, 15],
 [20240720, datetime.date(2024, 7, 20), 2024, 7, 20]]

In [65]:
customers_data

[['101', 'Ahmed Ali', 'M', 'Cairo'],
 ['102', 'Sara Hassan', 'F', 'Alexandria'],
 ['103', 'Omar Saeed', 'M', 'Giza'],
 ['104', 'Laila Mostafa', 'F', 'Cairo'],
 ['105', 'Hassan Ibrahim', 'M', 'Alexandria'],
 ['106', 'Mona Adel', 'F', 'Giza'],
 ['107', 'Tarek Hussein', 'M', 'Cairo'],
 ['108', 'Nour Youssef', 'F', 'Alexandria'],
 ['109', 'Karim Farouk', 'M', 'Giza'],
 ['110', 'Dina Mohamed', 'F', 'Cairo']]

In [74]:
cursor.executemany(
    "INSERT INTO dim_customer (customer_id, name, gender,city) VALUES (?, ?, ?,?)",
    customers_data
)

IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__dim_cust__CD65CB852682B9E6'. Cannot insert duplicate key in object 'dbo.dim_customer'. The duplicate key value is (101). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")

In [69]:
dates_data

[[20240510, datetime.date(2024, 5, 10), 2024, 5, 10],
 [20240511, datetime.date(2024, 5, 11), 2024, 5, 11],
 [20240512, datetime.date(2024, 5, 12), 2024, 5, 12],
 [20240601, datetime.date(2024, 6, 1), 2024, 6, 1],
 [20240605, datetime.date(2024, 6, 5), 2024, 6, 5],
 [20240610, datetime.date(2024, 6, 10), 2024, 6, 10],
 [20240702, datetime.date(2024, 7, 2), 2024, 7, 2],
 [20240705, datetime.date(2024, 7, 5), 2024, 7, 5],
 [20240715, datetime.date(2024, 7, 15), 2024, 7, 15],
 [20240720, datetime.date(2024, 7, 20), 2024, 7, 20]]

In [70]:
cursor.executemany(
    "INSERT INTO dim_date (date_key, full_date, year, month, day) VALUES (?, ?, ?, ?, ?)",
    dates_data
)
conn.commit()

In [76]:
fact_data

[[101, 201, 2, 20240510],
 [102, 202, 1, 20240511],
 [103, 203, 3, 20240512],
 [104, 204, 1, 20240601],
 [105, 205, 4, 20240605],
 [106, 206, 2, 20240610],
 [107, 207, 1, 20240702],
 [108, 208, 2, 20240705],
 [109, 209, 1, 20240715],
 [110, 210, 3, 20240720]]

In [75]:
cursor.executemany(
    """INSERT INTO fact_table (	product_id ,customer_id,quantity,date_key)
       VALUES (?, ?, ?, ?)""",
    fact_data
)

conn.commit()
cursor.close()
conn.close()

print("ETL Completed Successfully!")

IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert the value NULL into column 'fact_id', table 'CustomerDW.dbo.fact_table'; column does not allow nulls. INSERT fails. (515) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")